In [1]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
from data.info import (
    DataName,
    DataPath,
    STSDatasetFeatures,
    TCDatasetFeatures,
    TrainType,
    FileFormat,
    TrainType,
    UnsupervisedSimCseFeatures,
)
from datasets import load_from_disk

import logging

/home/cleaning/workspace/KoSimCSE/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
a = load_from_disk('data/datasets/train')


In [10]:
a

DatasetDict({
    dev: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1466
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1379
    })
})

In [7]:
b

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1466
})

In [8]:
c

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1379
})

In [ ]:
logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO
)


wiki_dataset = load_dataset("sh110495/kor-wikipedia")
#######################################################
data = []
for wiki_text in tqdm(wiki_dataset["train"]["text"]):
    wiki_text = wiki_text.replace(". ", ".\n")
    wiki_text = wiki_text.replace("\xa0", " ")
    wiki_sentences = wiki_text.split("\n")

    for wiki_sentence in wiki_sentences:
        wiki_sentence = wiki_sentence.rstrip().lstrip()
        if len(wiki_sentence) >= 10:
            data.append(wiki_sentence)
# data/utils.py로 옮기기
###########################################################
wiki_df = pd.DataFrame(data={UnsupervisedSimCseFeatures.SENTENCE.value: data})
wiki_df[UnsupervisedSimCseFeatures.SENTENCE.value] = wiki_df[
    UnsupervisedSimCseFeatures.SENTENCE.value
].apply(wiki_preprocess)


: 

In [ ]:


"""
sts_preprocess
"""

train_floder_path = get_folder_path(root=DataPath.ROOT, sub=DataPath.TRAIN)
dev_floder_path = get_folder_path(root=DataPath.ROOT, sub=DataPath.DEV)
test_floder_path = get_folder_path(root=DataPath.ROOT, sub=DataPath.TEST)

preprocess_wiki_data_path = get_data_path(
    folder_path=train_floder_path,
    data_source=DataName.PREPROCESS_WIKI,
    train_type=TrainType.TRAIN,
    file_format=FileFormat.CSV,
)
wiki_df = wiki_df.dropna(axis=0)
wiki_df.to_csv(preprocess_wiki_data_path, index=False)
# wiki_df = pd.read_csv(preprocess_wiki_data_path)
logging.info(
    f"preprocess wiki train done!\nfeatures:{wiki_df.columns} \nlen: {len(wiki_df)}\nna count:{sum(wiki_df[UnsupervisedSimCseFeatures.SENTENCE.value].isna())}"
)

klue_train = raw_data_to_dataframe(
    DataPath.ROOT, DataPath.RAW, DataName.RAW_KLUE, TrainType.TRAIN, FileFormat.JSON
)

kakao_train = raw_data_to_dataframe(
    DataPath.ROOT, DataPath.RAW, DataName.RAW_KAKAO, TrainType.TRAIN, FileFormat.TSV
)

kakao_dev = raw_data_to_dataframe(
    DataPath.ROOT, DataPath.RAW, DataName.RAW_KAKAO, TrainType.DEV, FileFormat.TSV
)

kakao_test = raw_data_to_dataframe(
    DataPath.ROOT, DataPath.RAW, DataName.RAW_KAKAO, TrainType.TEST, FileFormat.TSV
)
####
# tc
tc_train = raw_data_to_dataframe(
    DataPath.ROOT, DataPath.RAW, DataName.RAW_TC, TrainType.TRAIN, FileFormat.JSON
)
tc_dev = raw_data_to_dataframe(
    DataPath.ROOT, DataPath.RAW, DataName.RAW_TC, TrainType.DEV, FileFormat.JSON
)

tc_train = change_col_name(
    tc_train, TCDatasetFeatures.TITLE, UnsupervisedSimCseFeatures.SENTENCE
)
tc_dev = change_col_name(
    tc_dev, TCDatasetFeatures.TITLE, UnsupervisedSimCseFeatures.SENTENCE
)
print(tc_dev.head())
print(wiki_df.head())
#
####
unsup_datas = [wiki_df, tc_train, tc_dev]
total_sen = []
for unsup_data in unsup_datas:
    total_sen.extend(unsup_data[UnsupervisedSimCseFeatures.SENTENCE].to_list())
total_unsup_df = pd.DataFrame(data={UnsupervisedSimCseFeatures.SENTENCE: total_sen})

In [3]:
eval_data_files = {}
eval_data_files['dev'] = 'data/dev/sts_dev.tsv'
valid_dataset = load_dataset(
        "csv",
        data_files=eval_data_files,
        cache_dir="./data/.cache",
        delimiter="\t",
    )

04/18/2022 22:51:28 - WARNING - datasets.builder -   Using custom data configuration default-3b562c3513b8af77
04/18/2022 22:51:28 - WARNING - datasets.builder -   Reusing dataset csv (./data/.cache/csv/default-3b562c3513b8af77/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
for i in range(len(valid_dataset['dev'])):
    if not isinstance(valid_dataset['dev']['score'][i],float):
        print('dd',valid_dataset['dev']['score'][i],type(valid_dataset['dev']['score'][i]))

In [18]:
pearsonr = load_metric("pearsonr").compute
t = pearsonr(predictions=[1,1,2], references=[2,2,1])
t['pearsonr']

-1.0

In [2]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
from data.info import (
    DataName,
    DataPath,
    STSDatasetFeatures,
    TCDatasetFeatures,
    TrainType,
    FileFormat,
    TrainType,
    UnsupervisedSimCseFeatures,
)
from data.utils import (
    get_data_path,
    get_folder_path,
    raw_data_to_dataframe,
    make_unsupervised_sentence_data,
    wiki_preprocess,
    change_col_name,
    add_sts_df,
)

import logging

logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO
)


"""
wiki_preprocess
"""
wiki_dataset = load_dataset("sh110495/kor-wikipedia")
#######################################################
data = []
for wiki_text in tqdm(wiki_dataset["train"]["text"]):
    wiki_text = wiki_text.replace(". ", ".\n")
    wiki_text = wiki_text.replace("\xa0", " ")
    wiki_sentences = wiki_text.split("\n")

    for wiki_sentence in wiki_sentences:
        wiki_sentence = wiki_sentence.rstrip().lstrip()
        if len(wiki_sentence) >= 10:
            data.append(wiki_sentence)
# data/utils.py로 옮기기
###########################################################
wiki_df = pd.DataFrame(data={UnsupervisedSimCseFeatures.SENTENCE.value: data})
wiki_df[UnsupervisedSimCseFeatures.SENTENCE.value] = wiki_df[
    UnsupervisedSimCseFeatures.SENTENCE.value
].apply(wiki_preprocess)


"""
sts_preprocess
"""

train_floder_path = get_folder_path(root=DataPath.ROOT, sub=DataPath.TRAIN)
dev_floder_path = get_folder_path(root=DataPath.ROOT, sub=DataPath.DEV)
test_floder_path = get_folder_path(root=DataPath.ROOT, sub=DataPath.TEST)

preprocess_wiki_data_path = get_data_path(
    folder_path=train_floder_path,
    data_source=DataName.PREPROCESS_WIKI,
    train_type=TrainType.TRAIN,
    file_format=FileFormat.CSV,
)
wiki_df = wiki_df.dropna(axis=0)
wiki_df.to_csv(preprocess_wiki_data_path, index=False)
# wiki_df = pd.read_csv(preprocess_wiki_data_path)
logging.info(
    f"preprocess wiki train done!\nfeatures:{wiki_df.columns} \nlen: {len(wiki_df)}\nna count:{sum(wiki_df[UnsupervisedSimCseFeatures.SENTENCE.value].isna())}"
)

klue_train = raw_data_to_dataframe(
    DataPath.ROOT, DataPath.RAW, DataName.RAW_KLUE, TrainType.TRAIN, FileFormat.JSON
)

kakao_train = raw_data_to_dataframe(
    DataPath.ROOT, DataPath.RAW, DataName.RAW_KAKAO, TrainType.TRAIN, FileFormat.TSV
)

kakao_dev = raw_data_to_dataframe(
    DataPath.ROOT, DataPath.RAW, DataName.RAW_KAKAO, TrainType.DEV, FileFormat.TSV
)

kakao_test = raw_data_to_dataframe(
    DataPath.ROOT, DataPath.RAW, DataName.RAW_KAKAO, TrainType.TEST, FileFormat.TSV
)
####
# tc
tc_train = raw_data_to_dataframe(
    DataPath.ROOT, DataPath.RAW, DataName.RAW_TC, TrainType.TRAIN, FileFormat.JSON
)
tc_dev = raw_data_to_dataframe(
    DataPath.ROOT, DataPath.RAW, DataName.RAW_TC, TrainType.DEV, FileFormat.JSON
)

tc_train = change_col_name(
    tc_train, TCDatasetFeatures.TITLE, UnsupervisedSimCseFeatures.SENTENCE
)
tc_dev = change_col_name(
    tc_dev, TCDatasetFeatures.TITLE, UnsupervisedSimCseFeatures.SENTENCE
)
print(tc_dev.head())
print(wiki_df.head())
#
####
unsup_datas = [wiki_df, tc_train, tc_dev]
total_sen = []
for unsup_data in unsup_datas:
    total_sen.extend(unsup_data[UnsupervisedSimCseFeatures.SENTENCE].to_list())
total_unsup_df = pd.DataFrame(data={UnsupervisedSimCseFeatures.SENTENCE: total_sen})

In [3]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, OurTrainingArguments))

In [4]:
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if is_main_process(4) else logging.WARN,
    )

In [5]:
if is_main_process(4):
    transformers.utils.logging.set_verbosity_info()
    transformers.utils.logging.enable_default_handler()
    transformers.utils.logging.enable_explicit_format()
logger.info("Training/evaluation parameters %s")

04/16/2022 23:47:40 - INFO - __main__ -   Training/evaluation parameters %s


In [11]:
extension = "csv"
datasets = load_dataset(extension, data_files="data/train/wiki_train.csv", cache_dir="./data/")

04/16/2022 23:54:12 - WARNING - datasets.builder -   Using custom data configuration default-5b3a05775df14031


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to ./data/csv/default-5b3a05775df14031/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [13]:
model = BertForCL.from_pretrained("klue/bert-base",model_args=ModelArguments)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForCL: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForCL from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForCL from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForCL were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['mlp.de

In [14]:
model.resize_token_embeddings(len(tokenizer))

Embedding(32000, 768, padding_idx=0)

In [15]:
column_names = datasets["train"].column_names

In [20]:
column_names

['sentence']

In [21]:
sent0_cname = column_names[0]
sent1_cname = column_names[0]

In [22]:
sent0_cname

'sentence'

In [19]:
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence'],
        num_rows: 2861157
    })
})

In [23]:
examples = datasets['train']

In [29]:
sent2_cname = None
def prepare_features(examples):
    # padding = longest (default)
    #   If no sentence in the batch exceed the max length, then use
    #   the max sentence length in the batch, otherwise use the 
    #   max sentence length in the argument and truncate those that
    #   exceed the max length.
    # padding = max_length (when pad_to_max_length, for pressure test)
    #   All sentences are padded/truncated to data_args.max_seq_length.
    total = len(examples[sent0_cname])

    # Avoid "None" fields 
    for idx in range(total):
        if examples[sent0_cname][idx] is None:
            examples[sent0_cname][idx] = " "
        if examples[sent1_cname][idx] is None:
            examples[sent1_cname][idx] = " "
    
    sentences = examples[sent0_cname] + examples[sent1_cname]

    sent_features = tokenizer(
        sentences,
        max_length=32,
        truncation=True,
        padding="max_length",
    )

    features = {}
    if sent2_cname is not None:
        for key in sent_features:
            features[key] = [[sent_features[key][i], sent_features[key][i+total], sent_features[key][i+total*2]] for i in range(total)]
    else:
        for key in sent_features:
            features[key] = [[sent_features[key][i], sent_features[key][i+total]] for i in range(total)]
        
    return features

In [30]:
train_dataset = datasets["train"].map(prepare_features,batched=True,num_proc=4,remove_columns=column_names,load_from_cache_file=not DataTrainingArguments.overwrite_cache,)

#0:   0%|          | 0/716 [00:00<?, ?ba/s]

#1:   0%|          | 0/716 [00:00<?, ?ba/s]

#2:   0%|          | 0/716 [00:00<?, ?ba/s]

#3:   0%|          | 0/716 [00:00<?, ?ba/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2861157
})